In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

pd.set_option("display.max_columns", 300)

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [15]:
# In the data directory look for data bundles (directories with 2 csv files key_events.csv and landmarks.csv)
data_dir = "C:\\Users\\saman\\Dev\\AirKeyboardGUI\\out\\build\\x64-release\\AirKeyboardGUI\\logs"
data_bundles = [
    os.path.join(data_dir, d)
    for d in os.listdir(data_dir)
    if os.path.isdir(os.path.join(data_dir, d))
]

print(f"Found {len(data_bundles)} data bundles:\n" + "\n".join(data_bundles))

Found 16 data bundles:
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551910629445291
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551914799237503
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551915949449192
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551918126689860
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551921956411347
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551929922895738
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551933042424453
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551936781740118
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551940576670999
C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17578780212545489
C:\Users\saman\Dev\AirKeyboardGUI\out

In [16]:
def load_data(bundle_path):
    key_events_path = os.path.join(bundle_path, "key_events.csv")
    landmarks_path = os.path.join(bundle_path, "landmarks.csv")

    key_events = pd.read_csv(key_events_path)
    landmarks = pd.read_csv(landmarks_path)
    bundle_timestamp = os.path.basename(bundle_path).split("_")[-1]
    key_events["timestamp"] = key_events["timestamp"] + int(bundle_timestamp)
    landmarks["timestamp"] = landmarks["timestamp"] + int(bundle_timestamp)

    return key_events, landmarks


def check_left_right_mislabelling(landmarks):
    """Check if left and right hands are mislabeled by comparing wrist x-coordinates."""
    left_wrist = landmarks[(landmarks["hand_index"] == 0) & (landmarks["landmark_index"] == 0)]
    right_wrist = landmarks[(landmarks["hand_index"] == 1) & (landmarks["landmark_index"] == 0)]
    merged = pd.merge(
        left_wrist, right_wrist, on=["timestamp", "session_frame"], suffixes=("_left", "_right")
    )
    mislabeling = merged[merged["x_left"] > merged["x_right"]]
    return not mislabeling.empty


def correct_left_right_hands(landmarks):
    """Swap left and right hand labels."""
    landmarks["hand_label"] = landmarks["hand_label"].replace({"left": "tmp", "right": "left"})
    landmarks["hand_label"] = landmarks["hand_label"].replace({"tmp": "right"})

    landmarks["hand_index"] = landmarks["hand_index"].replace({0: -1, 1: 0})
    landmarks["hand_index"] = landmarks["hand_index"].replace({-1: 1})
    return landmarks


# Load all data bundles and concatenate them
all_key_events = []
all_landmarks = []
for bundle in data_bundles:
    ke, lm = load_data(bundle)

    if check_left_right_mislabelling(lm):
        print(f"Correcting left/right hand labels in bundle {bundle}")
        lm = correct_left_right_hands(lm)

    all_key_events.append(ke)
    all_landmarks.append(lm)

key_events = pd.concat(all_key_events, ignore_index=True)
landmarks = pd.concat(all_landmarks, ignore_index=True)

landmarks.drop(columns=["hand_score", "world_x", "world_y", "world_z"], inplace=True)


# landmark_index column
# 0. WRIST
# 1. THUMB_CMC
# 2. THUMB_MCP
# 3. THUMB_IP
# 4. THUMB_TIP
# 5. INDEX_FINGER_MCP
# 6. INDEX_FINGER_PIP
# 7. INDEX_FINGER_DIP
# 8. INDEX_FINGER_TIP
# 9. MIDDLE_FINGER_MCP
# 10. MIDDLE_FINGER_PIP
# 11. MIDDLE_FINGER_DIP
# 12. MIDDLE_FINGER_TIP
# 13. RING_FINGER_MCP
# 14. RING_FINGER_PIP
# 15. RING_FINGER_DIP
# 16. RING_FINGER_TIP
# 17. PINKY_MCP
# 18. PINKY_PIP
# 19. PINKY_DIP
# 20. PINKY_TIP

Correcting left/right hand labels in bundle C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17551940576670999
Correcting left/right hand labels in bundle C:\Users\saman\Dev\AirKeyboardGUI\out\build\x64-release\AirKeyboardGUI\logs\17578780212545489


In [17]:
# Subtract wrist coordinates from all landmarks to make them relative to the wrist
wrist_coords = landmarks[landmarks['landmark_index'] == 0][['timestamp', 'session_frame', 'hand_index', 'x', 'y', 'z']]
wrist_coords.rename(columns={'x': 'wrist_x', 'y': 'wrist_y', 'z': 'wrist_z'}, inplace=True)
landmarks = pd.merge(landmarks, wrist_coords, on=['timestamp', 'session_frame', 'hand_index'])
landmarks['x'] = landmarks['x'] - landmarks['wrist_x']
landmarks['y'] = landmarks['y'] - landmarks['wrist_y']
landmarks['z'] = landmarks['z'] - landmarks['wrist_z']
landmarks.drop(columns=['wrist_x', 'wrist_y', 'wrist_z'], inplace=True)

# Get the diffs of each joint between frames
landmarks.sort_values(by=["session_frame", "landmark_index"], inplace=True)
landmarks[["x_diff", "y_diff", "z_diff"]] = landmarks.groupby(["landmark_index", "hand_index"])[
    ["x", "y", "z"]
].diff()
landmarks.sort_values(by=["timestamp", "session_frame", "hand_index"], inplace=True)
landmarks

# # Standardize the diff columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
landmarks[['x', 'y', 'z']] = scaler.fit_transform(landmarks[['x', 'y', 'z']])

diff_scaler = StandardScaler()
landmarks[['x_diff', 'y_diff', 'z_diff']] = diff_scaler.fit_transform(landmarks[['x_diff', 'y_diff', 'z_diff']])


# Store the transformations for testing later
import joblib

joblib.dump(scaler, 'scaler.pkl')
joblib.dump(diff_scaler, 'diff_scaler.pkl')

landmarks

,session_frame,timestamp,hand_index,hand_label,landmark_index,x,y,z,x_diff,y_diff,z_diff
0,0,17551910634500752,0,left,0,0.139497,1.975921,2.094701,NaN,NaN,NaN
1,0,17551910634500752,0,left,1,0.856974,1.769521,2.107911,NaN,NaN,NaN
2,0,17551910634500752,0,left,2,1.430405,1.242191,1.896654,NaN,NaN,NaN
3,0,17551910634500752,0,left,3,1.633387,0.462944,1.810446,NaN,NaN,NaN
4,0,17551910634500752,0,left,4,1.612427,-0.110519,1.636308,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4798243,8681,17578807586533651,1,right,16,-1.052292,-0.723772,-1.438705,-0.785831,2.305740,0.076188
4798244,8681,17578807586533651,1,right,17,0.106902,0.649048,-0.180711,-0.442436,0.448687,0.108404
4798245,8681,17578807586533651,1,right,18,0.015495,0.012789,-0.891941,-0.732088,0.712566,0.152177
4798246,8681,17578807586533651,1,right,19,-0.153459,-0.372068,-1.211645,-1.001015,1.037187,0.278493


In [18]:
%%script false --no-raise-error

# Plot the distributions of x,y,z positions of all landmarks
plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.histplot(landmarks['x'], bins=50, kde=True, color='blue')
plt.title('Distribution of X Positions')
plt.xlabel('X Position')
plt.ylabel('Frequency')
plt.subplot(1, 3, 2)
sns.histplot(landmarks['y'], bins=50, kde=True, color='green')
plt.title('Distribution of Y Positions')
plt.xlabel('Y Position')
plt.ylabel('Frequency')
plt.subplot(1, 3, 3)
sns.histplot(landmarks['z'], bins=50, kde=True, color='red')
plt.title('Distribution of Z Positions')
plt.xlabel('Z Position')
plt.ylabel('Frequency')
plt.tight_layout()

# Plot the distributions of x_diff,y_diff,z_diff positions of all landmarks
plt.figure(figsize=(18, 5))
plt.subplot(2, 3, 1)
sns.histplot(landmarks['x_diff'], bins=50, kde=True, color='blue')
plt.title('Distribution of X Diff Positions')
plt.xlabel('X Diff Position')
plt.ylabel('Frequency')
plt.subplot(2, 3, 2)
sns.histplot(landmarks['y_diff'], bins=50, kde=True, color='green')
plt.title('Distribution of Y Diff Positions')
plt.xlabel('Y Diff Position')
plt.ylabel('Frequency')
plt.subplot(2, 3, 3)
sns.histplot(landmarks['z_diff'], bins=50, kde=True, color='red')
plt.title('Distribution of Z Diff Positions')
plt.xlabel('Z Diff Position')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


Couldn't find program: 'false'


In [19]:
# Pivot the landmarks dataframe to have one row per session_frame and hand_label, and columns for each landmark's x_diff, y_diff, z_diff
landmarks_pivot = landmarks.pivot_table(
    index=["timestamp", "session_frame"],
    columns=["hand_label", "landmark_index"],
    values=["x", "y", "z", "x_diff", "y_diff", "z_diff"],
)
landmarks_pivot.columns = [f"{hand}_{idx}_{axis}" for axis, hand, idx in landmarks_pivot.columns]
landmarks_pivot.reset_index(inplace=True)
landmarks_pivot

,timestamp,session_frame,left_0_x,left_1_x,left_2_x,left_3_x,left_4_x,left_5_x,left_6_x,left_7_x,left_8_x,left_9_x,left_10_x,left_11_x,left_12_x,left_13_x,left_14_x,left_15_x,left_16_x,left_17_x,left_18_x,left_19_x,left_20_x,right_0_x,right_1_x,right_2_x,right_3_x,right_4_x,right_5_x,right_6_x,right_7_x,right_8_x,right_9_x,right_10_x,right_11_x,right_12_x,right_13_x,right_14_x,right_15_x,right_16_x,right_17_x,right_18_x,right_19_x,right_20_x,left_0_x_diff,left_1_x_diff,left_2_x_diff,left_3_x_diff,left_4_x_diff,left_5_x_diff,left_6_x_diff,left_7_x_diff,left_8_x_diff,left_9_x_diff,left_10_x_diff,left_11_x_diff,left_12_x_diff,left_13_x_diff,left_14_x_diff,left_15_x_diff,left_16_x_diff,left_17_x_diff,left_18_x_diff,left_19_x_diff,left_20_x_diff,right_0_x_diff,right_1_x_diff,right_2_x_diff,right_3_x_diff,right_4_x_diff,right_5_x_diff,right_6_x_diff,right_7_x_diff,right_8_x_diff,right_9_x_diff,right_10_x_diff,right_11_x_diff,right_12_x_diff,right_13_x_diff,right_14_x_diff,right_15_x_diff,right_16_x_diff,right_17_x_diff,right_18_x_diff,right_19_x_diff,right_20_x_diff,left_0_y,left_1_y,left_2_y,left_3_y,left_4_y,left_5_y,left_6_y,left_7_y,left_8_y,left_9_y,left_10_y,left_11_y,left_12_y,left_13_y,left_14_y,left_15_y,left_16_y,left_17_y,left_18_y,left_19_y,left_20_y,right_0_y,right_1_y,right_2_y,right_3_y,right_4_y,right_5_y,right_6_y,right_7_y,right_8_y,right_9_y,right_10_y,right_11_y,right_12_y,right_13_y,right_14_y,right_15_y,right_16_y,right_17_y,right_18_y,right_19_y,right_20_y,left_0_y_diff,left_1_y_diff,left_2_y_diff,left_3_y_diff,left_4_y_diff,left_5_y_diff,left_6_y_diff,left_7_y_diff,left_8_y_diff,left_9_y_diff,left_10_y_diff,left_11_y_diff,left_12_y_diff,left_13_y_diff,left_14_y_diff,left_15_y_diff,left_16_y_diff,left_17_y_diff,left_18_y_diff,left_19_y_diff,left_20_y_diff,right_0_y_diff,right_1_y_diff,right_2_y_diff,right_3_y_diff,right_4_y_diff,right_5_y_diff,right_6_y_diff,right_7_y_diff,right_8_y_diff,right_9_y_diff,right_10_y_diff,right_11_y_diff,right_12_y_diff,right_13_y_diff,right_14_y_diff,right_15_y_diff,right_16_y_diff,right_17_y_diff,right_18_y_diff,right_19_y_diff,right_20_y_diff,left_0_z,left_1_z,left_2_z,left_3_z,left_4_z,left_5_z,left_6_z,left_7_z,left_8_z,left_9_z,left_10_z,left_11_z,left_12_z,left_13_z,left_14_z,left_15_z,left_16_z,left_17_z,left_18_z,left_19_z,left_20_z,right_0_z,right_1_z,right_2_z,right_3_z,right_4_z,right_5_z,right_6_z,right_7_z,right_8_z,right_9_z,right_10_z,right_11_z,right_12_z,right_13_z,right_14_z,right_15_z,right_16_z,right_17_z,right_18_z,right_19_z,right_20_z,left_0_z_diff,left_1_z_diff,left_2_z_diff,left_3_z_diff,left_4_z_diff,left_5_z_diff,left_6_z_diff,left_7_z_diff,left_8_z_diff,left_9_z_diff,left_10_z_diff,left_11_z_diff,left_12_z_diff,left_13_z_diff,left_14_z_diff,left_15_z_diff,left_16_z_diff,left_17_z_diff,left_18_z_diff,left_19_z_diff,left_20_z_diff,right_0_z_diff,right_1_z_diff,right_2_z_diff,right_3_z_diff,right_4_z_diff,right_5_z_diff,right_6_z_diff,right_7_z_diff,right_8_z_diff,right_9_z_diff,right_10_z_diff,right_11_z_diff,right_12_z_diff,right_13_z_diff,right_14_z_diff,right_15_z_diff,right_16_z_diff,right_17_z_diff,right_18_z_diff,right_19_z_diff,right_20_z_diff
0,17551910634500752,0,0.139497,0.856974,1.430405,1.633387,1.612427,1.129812,1.434990,1.574785,1.686546,0.649798,0.890503,1.047483,1.178130,0.249562,0.434812,0.615762,0.792354,-0.090545,0.028147,0.150795,0.305365,0.139497,-0.591893,-1.076822,-1.289163,-1.350623,-1.027002,-1.613814,-1.933804,-2.178513,-0.646867,-1.161032,-1.467642,-1.696835,-0.300919,-0.651711,-0.913107,-1.154964,0.016260,0.033029,-0.099923,-0.311753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.975921,1.769521,1.242191,0.462944,-0.110519,0.817696,-0.262452,-0.854989,-1.252428,0.627949,-0.607611,-1.256405,-1.635394,0.514061,-0.604808,-1.183261,-1.534091,0.489276,-0.440461,-0.930375,-1.272429,1.975921,1.782705,1.243918,0.5

In [ ]:
from scancode_mapping import SCANCODE_MAP


delay_ms = 37

# Create a copy of the landmarks timestamp with the delay removed.
landmarks_pivot["adjusted_timestamp"] = landmarks_pivot["timestamp"] - delay_ms

# Merge landmarks with key_events using merge_asof.
matched_frames = pd.merge_asof(
    landmarks_pivot.sort_values("adjusted_timestamp"),
    key_events.sort_values("timestamp"),
    left_on="adjusted_timestamp",
    right_on="timestamp",
    direction="nearest",
    suffixes=("", "_key"),
    tolerance=30,  # Only merge if within 30ms
)

matched_frames.sort_values(by=["timestamp", "session_frame"], inplace=True)
matched_frames["char"] = matched_frames["scancode"].map(SCANCODE_MAP).str.upper()


# Create an uppercase label consisting of the key and whether it was DOWN (pressed=1) or UP (pressed=0)
matched_frames["label"] = matched_frames.apply(
    lambda row: (
        f"{row['char']}_{'DOWN' if row['pressed'] == 1 else 'UP'}"
        if pd.notnull(row["char"])
        else "NO_KEY"
    ),
    axis=1,
)

# Drop rows where any coordinates are NaN (indicating missing hand data)
matched_frames = matched_frames.dropna(
    subset=[
        col for col in matched_frames.columns if any(axis in col for axis in ["_x", "_y", "_z"])
    ]
)

# Remove unnecessary columns
data = matched_frames.drop(
    columns=["timestamp_key", "adjusted_timestamp", "vkey", "scancode", "char"]
)
# Remove wrist columns
data = data.drop(columns=[col for col in data.columns if "_0" in col])
data

,timestamp,session_frame,left_1_x,left_2_x,left_3_x,left_4_x,left_5_x,left_6_x,left_7_x,left_8_x,left_9_x,left_10_x,left_11_x,left_12_x,left_13_x,left_14_x,left_15_x,left_16_x,left_17_x,left_18_x,left_19_x,left_20_x,right_1_x,right_2_x,right_3_x,right_4_x,right_5_x,right_6_x,right_7_x,right_8_x,right_9_x,right_10_x,right_11_x,right_12_x,right_13_x,right_14_x,right_15_x,right_16_x,right_17_x,right_18_x,right_19_x,right_20_x,left_1_x_diff,left_2_x_diff,left_3_x_diff,left_4_x_diff,left_5_x_diff,left_6_x_diff,left_7_x_diff,left_8_x_diff,left_9_x_diff,left_10_x_diff,left_11_x_diff,left_12_x_diff,left_13_x_diff,left_14_x_diff,left_15_x_diff,left_16_x_diff,left_17_x_diff,left_18_x_diff,left_19_x_diff,left_20_x_diff,right_1_x_diff,right_2_x_diff,right_3_x_diff,right_4_x_diff,right_5_x_diff,right_6_x_diff,right_7_x_diff,right_8_x_diff,right_9_x_diff,right_10_x_diff,right_11_x_diff,right_12_x_diff,right_13_x_diff,right_14_x_diff,right_15_x_diff,right_16_x_diff,right_17_x_diff,right_18_x_diff,right_19_x_diff,right_20_x_diff,left_1_y,left_2_y,left_3_y,left_4_y,left_5_y,left_6_y,left_7_y,left_8_y,left_9_y,left_10_y,left_11_y,left_12_y,left_13_y,left_14_y,left_15_y,left_16_y,left_17_y,left_18_y,left_19_y,left_20_y,right_1_y,right_2_y,right_3_y,right_4_y,right_5_y,right_6_y,right_7_y,right_8_y,right_9_y,right_10_y,right_11_y,right_12_y,right_13_y,right_14_y,right_15_y,right_16_y,right_17_y,right_18_y,right_19_y,right_20_y,left_1_y_diff,left_2_y_diff,left_3_y_diff,left_4_y_diff,left_5_y_diff,left_6_y_diff,left_7_y_diff,left_8_y_diff,left_9_y_diff,left_10_y_diff,left_11_y_diff,left_12_y_diff,left_13_y_diff,left_14_y_diff,left_15_y_diff,left_16_y_diff,left_17_y_diff,left_18_y_diff,left_19_y_diff,left_20_y_diff,right_1_y_diff,right_2_y_diff,right_3_y_diff,right_4_y_diff,right_5_y_diff,right_6_y_diff,right_7_y_diff,right_8_y_diff,right_9_y_diff,right_10_y_diff,right_11_y_diff,right_12_y_diff,right_13_y_diff,right_14_y_diff,right_15_y_diff,right_16_y_diff,right_17_y_diff,right_18_y_diff,right_19_y_diff,right_20_y_diff,left_1_z,left_2_z,left_3_z,left_4_z,left_5_z,left_6_z,left_7_z,left_8_z,left_9_z,left_10_z,left_11_z,left_12_z,left_13_z,left_14_z,left_15_z,left_16_z,left_17_z,left_18_z,left_19_z,left_20_z,right_1_z,right_2_z,right_3_z,right_4_z,right_5_z,right_6_z,right_7_z,right_8_z,right_9_z,right_10_z,right_11_z,right_12_z,right_13_z,right_14_z,right_15_z,right_16_z,right_17_z,right_18_z,right_19_z,right_20_z,left_1_z_diff,left_2_z_diff,left_3_z_diff,left_4_z_diff,left_5_z_diff,left_6_z_diff,left_7_z_diff,left_8_z_diff,left_9_z_diff,left_10_z_diff,left_11_z_diff,left_12_z_diff,left_13_z_diff,left_14_z_diff,left_15_z_diff,left_16_z_diff,left_17_z_diff,left_18_z_diff,left_19_z_diff,left_20_z_diff,right_1_z_diff,right_2_z_diff,right_3_z_diff,right_4_z_diff,right_5_z_diff,right_6_z_diff,right_7_z_diff,right_8_z_diff,right_9_z_diff,right_10_z_diff,right_11_z_diff,right_12_z_diff,right_13_z_diff,right_14_z_diff,right_15_z_diff,right_16_z_diff,right_17_z_diff,right_18_z_diff,right_19_z_diff,right_20_z_diff,pressed,label
1,17551910634500783,1,0.858401,1.369107,1.566431,1.576163,1.066320,1.357818,1.507251,1.622055,0.600038,0.807665,0.979118,1.137628,0.208568,0.347309,0.558921,0.776660,-0.110360,-0.046160,0.093499,0.282017,-0.575137,-1.069603,-1.296058,-1.328600,-1.015809,-1.643838,-1.932763,-2.133048,-0.617544,-1.172947,-1.488241,-1.733935,-0.242269,-0.654177,-0.927428,-1.172188,0.099403,0.050025,-0.097027,-0.295528,-0.035661,0.164088,0.338040,0.261080,0.156899,0.408423,0.404295,0.430317,0.225728,0.254786,0.123653,0.047783,0.211589,0.313221,0.276967,0.224525,0.148483,0.225987,0.151791,0.052165,-0.245092,-0.191502,-0.010326,0.223601,-0.017187,-0.075176,-0.004889,0.078316,0.234151,0.311057,0.360048,0.335692,0.341407,0.434660,0.495287,0.482890,0.324557,0.547259,0.669603,0.739663,1.746786,1.191151,0.446211,-0.130872,0.770998,-0.286924,-0.866354,-1.270456,0.583774,-0.630117,-1.254127,-1.635737,0.478853,-0.603605,-1.167932,-1.525413,0.477909,-0.418662,-0.8874

The input is of size 4x10x6, structured in the following format:
```plaintext
(pinky)      (ring)       (middle)    (index)      (thumb)     (thumb)      (index)      (middle)      (ring)        (pinky)       
left_20_t    left_16_t    left_12_t    left_8_t    left_4_t    right_4_t    right_8_t    right_12_t    right_16_t    right_20_t                
left_19_t    left_15_t    left_11_t    left_7_t    left_3_t    right_3_t    right_7_t    right_11_t    right_15_t    right_19_t           
left_18_t    left_14_t    left_10_t    left_6_t    left_2_t    right_2_t    right_6_t    right_10_t    right_14_t    right_18_t           
left_17_t    left_13_t    left_9 _t    left_5_t    left_1_t    right_1_t    right_5_t    right_9 _t    right_13_t    right_17_t      
```
where `t` is `_x | _y | _z | _x_diff | _y_diff | _z_diff` (6 channels) 

In [21]:
# Create one input sample of size 4x10x6 for each row in data
joint_cols = [
    col
    for col in data.columns
    if any(axis in col for axis in ["_x", "_y", "_z", "_x_diff", "_y_diff", "_z_diff"])
]

# Use to_numpy() for efficiency
input_values = data[joint_cols].to_numpy(dtype=float)

# Reshape the entire array at once
input_values = input_values.reshape(-1, 6, 10, 4)

# Concatenate along the second axis
input_values = np.concatenate(
    [input_values[:, :, :5, :][:, :, ::-1, :], input_values[:, :, 5:, :]], axis=2
)

# Assign the NumPy array to the 'data' column
data.drop(columns=joint_cols, inplace=True)
data.rename(columns={"session_frame": "frame_index", "label": "y"}, inplace=True)
data["X"] = [row.tolist() for row in input_values]

In [23]:
import numpy as np


KEY_POSITION_MAPPING = {
    "ESC": (0,0), "F1": (0,1), "F2": (0,2), "F3": (0,3), "F4": (0,4), "F5": (0,5), "F6": (0,6), "F7": (0,7), "F8": (0,8), "F9": (0,9), "F10": (0,10), "F11": (0,11), "F12": (0,12), "INS": (0,13), "DEL": (0,14),
    "`": (1,0), "1": (1,1), "2": (1,2), "3": (1,3), "4": (1,4), "5": (1,5), "6": (1,6), "7": (1,7), "8": (1,8), "9": (1,9), "0": (1,10), "-": (1,11), "=": (1,12), "BKSP": [(1,13), (1,14)],
    "TAB": (2,0), "Q": (2,1), "W": (2,2), "E": (2,3), "R": (2,4), "T": (2,5), "Y": (2,6), "U": (2,7), "I": (2,8), "O": (2,9), "P": (2,10), "[": (2,11), "]": (2,12), "ENTR": [(2,13), (2,14), (3,13), (3,14)],
    "CAPS": (3,0), "A": (3,1), "S": (3,2), "D": (3,3), "F": (3,4), "G": (3,5), "H": (3,6), "J": (3,7), "K": (3,8), "L": (3,9), ";": (3,10), "'": (3,11), "\\": [(3,12), (4,1)],
    "LSHFT": (4,0), "Z": (4,2), "X": (4,3), "C": (4,4), "V": (4,5), "B": (4,6), "N": (4,7), "M": (4,8), ",": (4,9), ".": (4,10), "/": (4,11), "RSHFT": [(4,12), (4,13), (4,14)],
    "LCTRL": (5,0), "FN": (5,1), "META": (5,2), "LALT": (5,3), "SPC": [(5,4), (5,5), (5,6), (5,7), (5,8)], "RALT": (5,9), "RCTRL": (5,10), "LEFT": (5,11), "UP": (5,12), "DOWN": (5,13), "RIGHT": (5,14)
}

# Map the labels to a one-hot matrix based on the KEY_POSITION_MAPPING
def multi_one_hot_normalize(label):
    key, action = label.split("_")
    one_hot = np.zeros((6, 15), dtype=int)
    if label == "NO_KEY":
        one_hot =  np.ones((6,15), dtype=int)  # Return a uniform distribution for NO_KEY
    positions = KEY_POSITION_MAPPING.get(key, [])
    if not isinstance(positions, list):
        positions = [positions]
    for pos in positions:
        one_hot[pos] = 1

    # Normalize and log-transform the one-hot matrix
    norm = np.linalg.norm(one_hot)
    if norm > 0:
        one_hot = one_hot / norm
        # Apply log transform with a small offset to avoid log(0)
        one_hot = np.log(one_hot + 1e-8)
        one_hot[np.isnan(one_hot)] = 0
        one_hot[np.isinf(one_hot)] = 0
    return one_hot

input_data = data.copy()
input_data["y1"] = input_data["y"].apply(multi_one_hot_normalize)
input_data["y2"] = input_data["pressed"].map(lambda x: [1, 0, 0] if x == 1 else ([0, 1, 0] if x == 0 else [0, 0, 1]))
input_data["y"] = input_data["y"].map(lambda x: x.split("_")[1] if x != "NO_KEY" else "NO_KEY")
input_data = input_data.drop(columns=["pressed"]).rename(columns={"y": "pressed_label"})

input_data.to_hdf(f"processed_data_{input_data['timestamp'].min()}_{input_data['timestamp'].max()}.h5", key='df', index=False)

C:\Users\saman\AppData\Local\Temp\ipykernel_26152\1266958015.py:41: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['pressed_label', 'X', 'y1', 'y2'], dtype='object')]

  input_data.to_hdf(f"processed_data_{input_data['timestamp'].min()}_{input_data['timestamp'].max()}.h5", key='df', index=False)
